In [1]:
# import the necessary packages
import imutils
import cv2
import numpy as np
# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread("neclace2.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
# threshold the image, then perform a series of erosions +
# dilations to remove any small regions of noise
#thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
#thresh = cv2.erode(thresh, None, iterations=2)
#thresh = cv2.dilate(thresh, None, iterations=2)
# find contours in thresholded image, then grab the largest
# one
ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)
ma = cv2.minAreaRect(c);
bp = cv2.boxPoints(ma)
#c1 = np.argpartition(c[c[:, :, 1]],3)
extRight = tuple(c[c[:, :, 0].argmin()][0])
extLeft = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])
extTop1 = tuple(bp[2])
print(extTop,extTop1)
#extTop1 = tuple(c[c[:, :, 1].argmin()][1])
#print(c1[1])
cv2.drawContours(image, [c], -1, (0, 255, 255), 2)
cv2.circle(image, extLeft, 8, (0, 0, 255), -1)
cv2.circle(image, extRight, 8, (0, 255, 0), -1)
cv2.circle(image, extTop, 8, (255, 0, 0), -1)
cv2.circle(image, extTop1, 8, (255, 0, 0), -1)
cv2.circle(image, extBot, 8, (255, 255, 0), -1)
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(50, 0) (385.0, 0.0)


In [36]:
c[c[:, :, 0].argmin()][0]

array([65, 58], dtype=int32)

In [54]:
index = np.argmin(c[:,:,1],axis = 1)


In [61]:
maximum_values = c[np.arange(c.shape[0]), index]


In [73]:
ma = cv2.minAreaRect(c);
bp = cv2.boxPoints(ma)
bp

array([[ 65., 350.],
       [ 65.,  28.],
       [314.,  28.],
       [314., 350.]], dtype=float32)

In [4]:
dis = bp[2][0]-extTop[0]
dis

240.0

In [70]:
h = int(bp[2][0]+20)
y = int(extTop[1]-20)
x = int(extTop[0]-20)
w = int(extBot[1]+20)
print(image.shape)
extTop
w

(380, 380, 3)


370

In [71]:
crop_img = image[y:w,x:h]
print(y,w,x,h)
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

8 370 54 334


In [8]:
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
def fit_factor(dim,dis):
    fac = round(dis/dim,2)
    return fac
def map_factor(img_dim,s2,tar_dim):
    if tar_dim[1]>img_dim[0]-(s2+80):
        new_dim = img_dim[0]-(s2+30)
    else:
        new_dim = tar_dim[1]
    return new_dim 
def acc_crop(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    ma = cv2.minAreaRect(c);
    bp = cv2.boxPoints(ma)
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    extTop1 = tuple(bp[2])
    h = int(bp[2][0]+20)
    y = int(extTop[1]-20)
    x = int(extTop[0]-20)
    w = int(extBot[1]+20)
    return x,y,w,h
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--shape-predictor", required=True,help="path to facial landmark predictor")
#ap.add_argument("-i", "--image", required=True,help="path to input image")
#args = vars(ap.parse_args())
cap = cv2.VideoCapture(0)
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
image1 = cv2.imread('neclace5.jpg') 
#x,y,w,h = acc_crop(oimage)
#image1 = oimage[y:w,x:h]
while True:
    ret, image = cap.read() 
    # load the input image, resize it, and convert it to grayscale
    #image = cv2.imread('neck.jpg')
    #print(image.shape)
    #image = imutils.resize(image, width=500)
    #print(image.shape)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        #for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        name = 'jaw'
        (i, j) = face_utils.FACIAL_LANDMARKS_IDXS[name]
        (i, j) = (i+3,j-4)
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone,'neck', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        (s1,s2) = shape[i]
        (e1,e2) = shape[j]
        dis = e1-s1
        dim = image1.shape[1]
        print((s1,s2), (e1,e2))
        print(e1-s1)
        print(image1.shape)
        m_factor = fit_factor(dim,dis)
        p_dim = (int(image1.shape[1]*m_factor),int(image1.shape[0]*m_factor))
        print(p_dim)
        mf = map_factor(image.shape,s2,p_dim)
        if mf ==p_dim[0]:
            dim = p_dim
        else:
            dim = (p_dim[0],mf)
        print(dim)
        img2gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

        # add a threshold
        ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('Ear Detector', ear_img) 
        mask_inv = cv2.bitwise_not(mask)
        resized = cv2.resize(image1,dim, interpolation = cv2.INTER_AREA)
        #print(resized.shape)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask.shape)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask_inv.shape)
        resized1 = cv2.resize(resized, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask.shape)
        mask_inv = cv2.resize(mask_inv, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask_inv.shape)
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y+40), 3, (0, 255, 0), -1)

        # extract the ROI of the face region as a separate image
        #(x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        #roi = image[y:y + h, x:x + w]
        #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        print(x, y, w, h)
        point = s1
        point1 = s2+30
        roi1 = image[point1:point1+dim[1],point:point+dim[0]]
        print(roi1.shape)
        print(point1+dim[1],point+dim[0])
        roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(s1,point,point1,point1+dim[0])
        print(roi.shape)
        print(point,point1)
        roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
        print(roi_bg.shape)
        # roi_fg contains the orignal location of earring
        roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
        print(roi_bg.shape)
        # joining the roi_bg and roi_fg
        dst1 = cv2.add(roi_bg,roi_fg)
        dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
        print(dst.shape)
        #cv2.imshow('Ear Detector', dst) 
        #cv2.waitKey(10)
        #point = int(s1+dim[0])
        #print(point,point+dim[0])
        cv2.circle(image, (s1,s2+80), 3, (0, 255, 0), -1)
        #cv2.rectangle(image, (s1,s2+80), (s1+dim[0],s2+80+dim[1]), (0,255,0), 3)
        image[point1:point1+dim[1],point:point+dim[0]]=dst
        # show the particular face part
        #cv2.imshow("ROI", roi)
        cv2.imshow("Image", image)
        #cv2.waitKey(0)
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    k = cv2.waitKey(0) & 0xFF
    if k == ord('q'): # wait for ESC key to exit
        break
    #if cv2.waitKey(0) & 0xFF == ord('s'):
        #break

cap.release() 
cv2.destroyAllWindows()

(250, 270) (371, 261)
121
(500, 500, 3)
(120, 120)
(120, 120)
(1200, 1200)
(1200, 1200)
250 270 120 37
(120, 120, 3)
420 370
250 250 300 420
(1200, 1200, 3)
250 300
(1200, 1200, 3)
(1200, 1200, 3)
(120, 120, 3)


In [1]:
# import the necessary packages
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import urllib.request
def fit_factor(dim,dis):
    fac = round(dis/dim,2)
    return fac
def map_factor(img_dim,s2,tar_dim):
    if tar_dim[1]>img_dim[0]-(s2+80):
        new_dim = img_dim[0]-(s2+10)
    else:
        new_dim = tar_dim[1]
    return new_dim  
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--shape-predictor", required=True,help="path to facial landmark predictor")
#ap.add_argument("-i", "--image", required=True,help="path to input image")
#args = vars(ap.parse_args())
cap = cv2.VideoCapture(0)
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
#image1 = cv2.imread('neclace4.jpg')
oimage = cv2.imread('neclace4.jpg') 
y,w,x,h = 8,370,54,334
URL = "http://192.168.43.1:8080/shot.jpg"
while True:
    #ret, image = cap.read() 
    # load the input image, resize it, and convert it to grayscale
    #image = cv2.imread('gouse1.jpg')
    #print(image.shape)
    #image = imutils.resize(image, width=500)
    #print(image.shape)
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    #print(img_arr)
    image = cv2.imdecode(img_arr,-1)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        #for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        name = 'jaw'
        (i, j) = face_utils.FACIAL_LANDMARKS_IDXS[name]
        (i, j) = (i+3,j-4)
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone,'neck', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        (s1,s2) = shape[i]
        (e1,e2) = shape[j]
        dis = e1-s1
        dim = image1.shape[1]
        print((s1,s2), (e1,e2))
        print(e1-s1)
        print(image1.shape)
        m_factor = fit_factor(dim,dis)
        p_dim = (int(image1.shape[1]*m_factor),int(image1.shape[0]*m_factor))
        print(p_dim)
        mf = map_factor(image.shape,s2,p_dim)
        if mf ==p_dim[0]:
            dim = p_dim
        else:
            dim = (p_dim[0],mf)
        print(dim)
        img2gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

        # add a threshold
        ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('Ear Detector', ear_img) 
        mask_inv = cv2.bitwise_not(mask)
        resized = cv2.resize(image1,dim, interpolation = cv2.INTER_AREA)
        #print(resized.shape)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask.shape)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask_inv.shape)
        resized1 = cv2.resize(resized, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask.shape)
        mask_inv = cv2.resize(mask_inv, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask_inv.shape)
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y+40), 3, (0, 255, 0), -1)

        # extract the ROI of the face region as a separate image
        #(x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        #roi = image[y:y + h, x:x + w]
        #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        print(x, y, w, h)
        point = s1
        point1 = s2+50
        roi1 = image[point1:point1+dim[1],point:point+dim[0]]
        print(roi1.shape)
        print(point1+dim[1],point+dim[0])
        roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(s1,point,point1,point1+dim[0])
        print(roi.shape)
        print(point,point1)
        roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
        print(roi_bg.shape)
        # roi_fg contains the orignal location of earring
        roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
        print(roi_bg.shape)
        # joining the roi_bg and roi_fg
        dst1 = cv2.add(roi_bg,roi_fg)
        dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
        print(dst.shape)
        #cv2.imshow('Ear Detector', dst) 
        #cv2.waitKey(10)
        #point = int(s1+dim[0])
        #print(point,point+dim[0])
        cv2.circle(image, (s1,s2+80), 3, (0, 255, 0), -1)
        #cv2.rectangle(image, (s1,s2+80), (s1+dim[0],s2+80+dim[1]), (0,255,0), 3)
        image[point1:point1+dim[1],point:point+dim[0]]=dst
        # show the particular face part
        #cv2.imshow("ROI", roi)
        cv2.imshow("Image", image)
        #cv2.waitKey(0)
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    k = cv2.waitKey(0) & 0xFF
    if k == ord('q'): # wait for ESC key to exit
        break
    #if cv2.waitKey(0) & 0xFF == ord('s'):
        #break

cap.release() 
cv2.destroyAllWindows()

URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>

In [6]:
def acc_crop(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    ma = cv2.minAreaRect(c);
    bp = cv2.boxPoints(ma)
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    extTop1 = tuple(bp[2])
    h = int(bp[2][0]+20)
    y = int(extTop[1]-20)
    x = int(extTop[0]-20)
    w = int(extBot[1]+20)
    return x,y,w,h


In [11]:
oimage = cv2.imread("neclace7.jpg")
print(oimage.shape)
#x,y,w,h = acc_crop(oimage)
#print(x,y,w,h)
first_half = oimage[0:oimage.shape[1],0:int(oimage.shape[0]/2)]
second_half = oimage[0:oimage.shape[1],int(oimage.shape[0]/2):oimage.shape[0]]
print(int(oimage.shape[0]/2),oimage.shape[0])
#cv2.imshow("cropped", first_half)
cv2.imshow("cropped", second_half)
cv2.waitKey(0)
cv2.destroyAllWindows()

(700, 700, 3)
350 700


In [14]:
def acc_crop_side(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    ma = cv2.minAreaRect(c);
    bp = cv2.boxPoints(ma)
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    return extTop
    

In [26]:
def acc_crop_bottom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    return extBot,extRight,extLeft

In [21]:
topl = acc_crop_side(first_half)
topr = acc_crop_side(second_half)
topl,topr

((102, 40), (245, 47))

In [38]:
dis = (int(oimage.shape[0]/2)-lf[0])+rf[0]

In [27]:
bot,right,left = acc_crop_bottom(oimage)
bot,right,left

((340, 681), (606, 92), (96, 76))

In [33]:
dis = (int(oimage.shape[0]/2)+rf[0])
if topl[1]>topr[1]:
    y = topr[1]
else:
    y = topl[1]
if topl[0]>left[0]:
    x = left[0]
else:
    x = topl[0]
if dis>right[0]:
    w = dis
else:
    w = right[0]
h = bot[1]
cor_points = [x-10,y-10,w+10,h+10]
final_cor_points = [x if x>0 else x-20 for x in cor_points]
final_cor_points

[86, 30, 616, 691]

In [19]:
h1 = dis+20
y1 = lf[1]-20
x1 = lf[0]-20
w1 = bot[1]+10
cor_points = [h1,y1,x1,w1]
final_cor_points = [x if x>0 else x-20 for x in cor_points]
final_cor_points

NameError: name 'dis' is not defined

In [42]:
def acc_cor_points(oimage):
    first_half = oimage[0:oimage.shape[1],0:int(oimage.shape[0]/2)]
    second_half = oimage[0:oimage.shape[1],int(oimage.shape[0]/2):oimage.shape[0]]
    lf = acc_crop_side(first_half)
    rf = acc_crop_side(second_half)
    dis = (int(oimage.shape[0]/2)-lf[0])+rf[0]
    bot,right,left = acc_crop_bottom(oimage)
    dis = (int(oimage.shape[0]/2)+rf[0])
    if topl[1]>topr[1]:
        y = topr[1]
    else:
        y = topl[1]
    if topl[0]>left[0]:
        x = left[0]
    else:
        x = topl[0]
    if dis>right[0]:
        w = dis
    else:
        w = right[0]
    h = bot[1]
    #cor_points = [x-10,y-10,w+10,h+10]
    #cor_points = [x,y,w,h]
    #final_cor_points = [x if x>0 else x-20 for x in cor_points]
    return x,y,w,h

In [43]:
print(acc_cor_points(oimage))

(96, 40, 606, 681)


In [11]:
oimage = cv2.imread('neclace1.jpg') 
h,y,x,w = acc_cor_points(oimage)
print(h,y,x,w)
image1 = oimage[y:w,int(x/2):int(x*1.5)+h]
cv2.imshow("cropped", image1)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'acc_cor_points' is not defined

In [ ]:
extRight = tuple(c[c[:, :, 0].argmin()][0])
extLeft = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])
extTop1 = tuple(bp[2])

In [9]:
extTop,extLeft,extRight,extBot

((50, 0), (385, 60), (28, 69), (215, 337))